In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
items1 = pd.read_csv('../data/item_properties_part1.csv')
items2 = pd.read_csv('../data/item_properties_part2.csv')

In [3]:
items = pd.concat([items1, items2])
items.head(10)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
5,1436065200000,285026,available,0
6,1434250800000,89534,213,1121373
7,1431831600000,264312,6,319724
8,1433646000000,229370,202,1330310
9,1434250800000,98113,451,1141052 n48.000


In [4]:
import datetime
times =[]
for i in items['timestamp']:
    times.append(datetime.datetime.fromtimestamp(i//1000.0)) 
items['timestamp'] = times
items.head(10)

,timestamp,itemid,property,value
0,2015-06-28 05:00:00,460429,categoryid,1338
1,2015-09-06 05:00:00,206783,888,1116713 960601 n277.200
2,2015-08-09 05:00:00,395014,400,n552.000 639502 n720.000 424566
3,2015-05-10 05:00:00,59481,790,n15360.000
4,2015-05-17 05:00:00,156781,917,828513
5,2015-07-05 05:00:00,285026,available,0
6,2015-06-14 05:00:00,89534,213,1121373
7,2015-05-17 05:00:00,264312,6,319724
8,2015-06-07 05:00:00,229370,202,1330310
9,2015-06-14 05:00:00,98113,451,1141052 n48.000


In [5]:
events = pd.read_csv('../data/events.csv')
events.head(10)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
5,1433224086234,972639,view,22556,NaN
6,1433221923240,810725,view,443030,NaN
7,1433223291897,794181,view,439202,NaN
8,1433220899221,824915,view,428805,NaN
9,1433221204592,339335,view,82389,NaN


In [6]:
category_tree = pd.read_csv('../data/category_tree.csv')
category_tree.head(10)

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
5,231,NaN
6,542,378.0
7,1146,542.0
8,1140,542.0
9,1479,1537.0


In [7]:
events = events.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in events.timestamp))
events = events.sort_values('date').reset_index(drop=True)
events = events[['visitorid','itemid','event', 'date']]
events.head(5)

,visitorid,itemid,event,date
0,1155627,61279,view,2015-05-03
1,53842,12217,view,2015-05-03
2,710338,81856,view,2015-05-03
3,807228,395446,view,2015-05-03
4,190672,259357,transaction,2015-05-03


In [8]:
start_date = '2015-5-3'
end_date = '2015-5-18'
fd = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date()
events = events[(events.date >= fd(start_date)) & (events.date <= fd(end_date))]

In [9]:
split_point = np.int(np.round(events.shape[0]*0.8))
events_train = events.iloc[0:split_point]
events_test = events.iloc[split_point::]
#Get only visitors and items that are in train as well
#This is for testing so we have true positives
events_test = events_test[
    (events_test['visitorid'].isin(events_train['visitorid'])) & 
    (events_test['itemid'].isin(events_train['itemid']))
]
print(events_train.shape)
print(events_test.shape)
# (64019, 4)
# (10510, 4)

(256076, 4)
(10510, 4)


In [10]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from scipy.sparse import coo_matrix
from sklearn import preprocessing

/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [120]:
id_cols=['visitorid','itemid']
trans_cat_train=dict()
trans_cat_test=dict()

cate_enc_dict = dict()
for k in id_cols:
    cate_enc_dict[k] = preprocessing.LabelEncoder()
    trans_cat_train[k]=cate_enc_dict[k].fit_transform(events_train[k].values)
    trans_cat_test[k]=cate_enc_dict[k].transform(events_test[k].values)

# trans_cat_train=dict()
# trans_cat_test=dict()
# items_index = dict()
# cate_enc=preprocessing.LabelEncoder()
# trans_cat_train['visitorid']=cate_enc.fit_transform(events_train['visitorid'].values)
# trans_cat_test['visitorid']=cate_enc.transform(events_test['visitorid'].values)
# items_index['itemid']=cate_enc.fit_transform(np.unique(events['itemid']))

In [121]:
ratings = dict()
cate_enc_dict['interact']=preprocessing.LabelEncoder()
ratings['train'] = cate_enc_dict['interact'].fit_transform(events_train.event)
ratings['test'] = cate_enc_dict['interact'].transform(events_test.event)


In [122]:
np.unique(ratings['train'],return_counts=True)

(array([0, 1, 2]), array([  6518,   2103, 247455]))

In [123]:
n_users=len(np.unique(trans_cat_train['visitorid']))
n_items=len(np.unique(trans_cat_train['itemid']))
rate_matrix = dict()
rate_matrix['train'] = coo_matrix((ratings['train'], (trans_cat_train['visitorid'], trans_cat_train['itemid'])), shape=(n_users,n_items))
rate_matrix['test'] = coo_matrix((ratings['test'], (trans_cat_test['visitorid'], trans_cat_test['itemid'])), shape=(n_users,n_items))


In [193]:
model = LightFM(no_components=10, loss='warp')
model.fit(rate_matrix['train'], epochs=100, num_threads=8)

In [49]:
print("Train:AUC:",auc_score(model, rate_matrix['train'], num_threads=8).mean())
print("Test :AUC:",auc_score(model, rate_matrix['test'], num_threads=10).mean())
print("Train:Prec:",precision_at_k(model, rate_matrix['train'], num_threads=8).mean())
print("Test :Prec:",precision_at_k(model, rate_matrix['test'], num_threads=10).mean())

Train:AUC: 0.99861
Test :AUC: 0.8225744
Train:Prec: 0.045533966
Test :Prec: 0.030359356


In [97]:
print("Train:Rcll:",recall_at_k(model, rate_matrix['train'], num_threads=8).mean())
print("Test :Rcll:",recall_at_k(model, rate_matrix['test'], num_threads=10).mean())

Train:Rcll: 0.23458518928291955
Test :Rcll: 0.27155334744408816


In [310]:
def predict_recom(user_id_orig, recom_num, model):
    result_dict = dict()
    result_array = list()
    user_id = cate_enc_dict['visitorid'].transform([user_id_orig])[0]
    #Known Positives
    known_positives = np.unique(np.concatenate((events_train['itemid'].values[np.where(trans_cat_train['visitorid']==user_id)]
                    , events_test['itemid'].values[np.where(trans_cat_test['visitorid']==user_id)]), axis=0))
    
    #Recommended
    scores = np.unique(model.predict(user_ids = [user_id], item_ids = trans_cat_test['itemid']))
    scores_list = dict()
#     for score in np.argsort((-np.unique(scores))):
#         scores_list['score'] = score
#         scores_list['item_id'] = trans_cat_train['itemid'][score]
#     print("score:",score)
#     print(trans_cat_train['itemid'][score])

    top_items = trans_cat_test['itemid'][np.argsort(-scores)]
    print("TOP",len(scores))
    print(len(trans_cat_test['itemid']))

    # printing out the result
    print("User %s" % cate_enc_dict['visitorid'].inverse_transform([user_id])[0])
    print("     Known positives:")

    for x in known_positives[:recom_num]:
        print("                  %s" % x)


    print("     Recommended:")

    for x in top_items[:recom_num]:
        print("                  %s" % cate_enc_dict['itemid'].inverse_transform([x])[0])
        result_array.append(x)
    results_dict[user_id_orig] = result_array
    print("")
    return results_dict
    

In [311]:
for usr in np.array(events_test['visitorid'])[:2]:
    top_rec = predict_recom(usr, 5, model)
print(top_rec)

model.fit_partial(rate_matrix['test'], epochs=100, num_threads=8)
print("Added knowledge to model....✅")

TOP 5289
10510
User 275412
     Known positives:
                  315545
                  320494
                  370360
     Recommended:
                  95729
                  143418
                  340749
                  304685
                  34714

TOP 5290
10510
User 113285
     Known positives:
                  178430
                  252769
                  258856
                  276240
                  277380
     Recommended:
                  77606
                  95729
                  143418
                  340749
                  304685

{275412: [14677, 22027, 52375, 46882, 5274], 113285: [11844, 14677, 22027, 52375, 46882], 595799: [14677, 22027], 132397: [11844, 14677], 995772: [11844, 14677]}
Added knowledge to model....✅


In [316]:
# cate_enc_dict['visitorid'].inverse_transform([57418])
# cate_enc_dict['itemid'].inverse_transform([63480])
events[
    (events['itemid']==95729)
#     &   (events['visitorid']==113285)
#        | (events['visitorid']==171109)
       
      ].sort_values(by=['visitorid','itemid'])

,visitorid,itemid,event,date
179951,55389,95729,view,2015-05-12
276647,120925,95729,view,2015-05-17
245615,170599,95729,view,2015-05-15
279440,286616,95729,view,2015-05-17
152786,566137,95729,view,2015-05-11
183860,713402,95729,view,2015-05-13
189249,713402,95729,addtocart,2015-05-13
190757,713402,95729,view,2015-05-13
189424,781655,95729,view,2015-05-13
114381,809822,95729,view,2015-05-08


In [269]:
trans_cat_train['itemid'][71707]

45311